In [82]:
import torch
from torch import nn
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.models import ResNet18_Weights

In [83]:
resnet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [85]:
data_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    #transforms.RandomHorizontalFlip(p=0.7),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
]) 

In [86]:
import torch
import torchvision.transforms as transforms
from torchvision.transforms import InterpolationMode
import random

class RandomGammaAdjustment:
    def __init__(self, gamma_range=(0.8, 1.2)):
        self.gamma_range = gamma_range

    def __call__(self, img):
        gamma = random.uniform(*self.gamma_range)
        return transforms.functional.adjust_gamma(img, gamma, gain=1)

    def __repr__(self):
        return f"{self.__class__.__name__}(gamma_range={self.gamma_range})"

data_aug = transforms.Compose([
    transforms.Resize(size=(128, 128), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.7),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=60),
    transforms.RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.3),
    RandomGammaAdjustment(gamma_range=(0.6, 1.3)),
    transforms.ToTensor(),
])


In [87]:
import os
from torchvision import datasets, transforms

train_data_path = "/kaggle/input/d/chferra/lung-ds-mod/Nodule/train"
validation_data_path = "/kaggle/input/d/chferra/lung-ds-mod/Nodule/val"

train_data = datasets.ImageFolder(root=train_data_path, transform=data_aug)
validation_data = datasets.ImageFolder(root=validation_data_path, transform=data_transform)

In [88]:
train_data

Dataset ImageFolder
    Number of datapoints: 1888
    Root location: /kaggle/input/d/chferra/lung-ds-mod/Nodule/train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.7)
               RandomVerticalFlip(p=0.5)
               RandomRotation(degrees=[-60.0, 60.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.9, 1.1), interpolation=bilinear, antialias=True)
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.7, 1.3), saturation=None, hue=None)
               RandomGammaAdjustment(gamma_range=(0.6, 1.3))
               ToTensor()
           )

In [89]:
class_names = train_data.classes
class_names

['0', '1', '2', '3', '4']

In [90]:
class_dict = train_data.class_to_idx
class_dict

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [91]:
BATCH_SIZE=32
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=1,
                              shuffle=True)

val_dataloader = DataLoader(dataset=validation_data,
                             batch_size=BATCH_SIZE,
                             num_workers=1,
                             shuffle=False)

train_dataloader, val_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7e2cc6367df0>,
 <torch.utils.data.dataloader.DataLoader at 0x7e2cc6367af0>)

In [92]:
resnet18.fc = nn.Sequential (
    nn.Linear(512,512),
    nn.Dropout(0.2),
    nn.Linear(512,256),
    nn.Linear(256,128),
    nn.Linear(128,64),
    nn.Linear(64,len(class_names)),
)
resnet18.fc

Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Dropout(p=0.2, inplace=False)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): Linear(in_features=64, out_features=5, bias=True)
)

In [93]:
resnet18.conv1 = nn.Conv2d(3, 
                           64, 
                           kernel_size=(7, 7), 
                           stride=(2, 2), 
                           padding=(3, 3), 
                           bias=False)

In [94]:
from collections import Counter

def compute_class_weights(dataloader):
    all_labels = []
    for _, y in dataloader:
        all_labels.extend(y.numpy())
    
    class_counts = Counter(all_labels)
    total_samples = sum(class_counts.values())
    num_classes = len(class_counts)
    
    class_weights = {cls: total_samples / (num_classes * count) for cls, count in class_counts.items()}

    weight_tensor = torch.tensor([class_weights[cls] for cls in range(num_classes)], dtype=torch.float32)
    return weight_tensor

In [95]:
from torch.optim import lr_scheduler

class_weights = compute_class_weights(train_dataloader).to(device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(resnet18.parameters(), lr=1e-4)

class_weights

tensor([1.9364, 1.0345, 0.4325, 1.1305, 3.1207])

In [75]:
from torchinfo import summary
summary(resnet18, input_size=[1, 3, 64, 64])

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 5]                    --
├─Conv2d: 1-1                            [1, 64, 32, 32]           9,408
├─BatchNorm2d: 1-2                       [1, 64, 32, 32]           128
├─ReLU: 1-3                              [1, 64, 32, 32]           --
├─MaxPool2d: 1-4                         [1, 64, 16, 16]           --
├─Sequential: 1-5                        [1, 64, 16, 16]           --
│    └─BasicBlock: 2-1                   [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-1                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-3                    [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-4                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-6                    [1, 64, 16, 16]           --
│

In [96]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X) 


    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

  
    optimizer.zero_grad()

    
    loss.backward()

  
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)
  
  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader) 
  return train_loss, train_acc 


In [97]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
  model.eval()

  test_loss, test_acc = 0,  0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader): 
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)

     
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

    
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [98]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader,
          val_dataloader,
          optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5, 
          device=device):
  
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}
  
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=val_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)
    
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  
  return results

In [99]:
NUM_EPOCHS = 15

from timeit import default_timer as timer
start_time = timer() 

model_0_results = train(model=resnet18,
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 1.4204 | Train acc: 0.2818 | Test loss: 1.3289 | Test acc: 0.3230
Epoch: 1 | Train loss: 1.2288 | Train acc: 0.4243 | Test loss: 1.3874 | Test acc: 0.3572
Epoch: 2 | Train loss: 1.1881 | Train acc: 0.4269 | Test loss: 1.3212 | Test acc: 0.3947
Epoch: 3 | Train loss: 1.1378 | Train acc: 0.4544 | Test loss: 1.2453 | Test acc: 0.4267
Epoch: 4 | Train loss: 1.1126 | Train acc: 0.4417 | Test loss: 1.2322 | Test acc: 0.4051
Epoch: 5 | Train loss: 1.0906 | Train acc: 0.4507 | Test loss: 1.2087 | Test acc: 0.4452
Epoch: 6 | Train loss: 1.0766 | Train acc: 0.4613 | Test loss: 1.2395 | Test acc: 0.4460
Epoch: 7 | Train loss: 1.0877 | Train acc: 0.4804 | Test loss: 1.2054 | Test acc: 0.4524
Epoch: 8 | Train loss: 1.0564 | Train acc: 0.4719 | Test loss: 1.2593 | Test acc: 0.4249
Epoch: 9 | Train loss: 1.0379 | Train acc: 0.4502 | Test loss: 1.2078 | Test acc: 0.4278
Epoch: 10 | Train loss: 1.0226 | Train acc: 0.4793 | Test loss: 1.2176 | Test acc: 0.4873
Epoch: 11 | Train lo